In [141]:
from bs4 import BeautifulSoup
import requests
import time, os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

import pandas as pd


In [153]:
# Bringing in Data via Web Scrapping
#url_2017 ='https://www.baseball-reference.com/teams/tgl.cgi?team=NYY&t=b&year=2017'
url_2016 = 'https://www.baseball-reference.com/teams/NYY/2016.shtml'
df_2016  = pd.read_html(url_2016)[0]

url_2017 = "https://www.baseball-reference.com/teams/NYY/2017.shtml" # Da Yankees
df_2017 = pd.read_html(url_2017)[0]

url_2018 = "https://www.baseball-reference.com/teams/NYY/2018.shtml"
df_2018 = pd.read_html(url_2018)[0]

url_2019 = "https://www.baseball-reference.com/teams/NYY/2019.shtml"
df_2019 = pd.read_html(url_2019)[0]


In [154]:
#  clean 2016dropped 'Rk' Column inplace true and 9th row because it was another column header. reduced to just batters added column for year visual
df_2016.drop('Rk',axis = 1, inplace = True)
df_2016.drop([9],inplace = True)
df_2016["YEAR"] =('2016')
clean_df_2016 = (df_2016[df_2016.index < 25])

clean_df_2016.head(10)


,Pos,Name,Age,G,PA,AB,R,H,2B,3B,...,SLG,OPS,OPS+,TB,GDP,HBP,SH,SF,IBB,YEAR
0,C,Brian McCann*,32,130,492,429,56,104,13,0,...,.413,.748,99,177,15,7,0,2,2,2016
1,1B,Mark Teixeira#,36,116,438,387,43,79,16,0,...,.362,.654,74,140,7,2,0,2,1,2016
2,2B,Starlin Castro,26,151,610,577,63,156,29,1,...,.433,.734,93,250,15,3,1,5,1,2016
3,SS,Didi Gregorius*,26,153,597,562,68,155,32,2,...,.447,.751,97,251,9,6,5,5,2,2016
4,3B,Chase Headley#,32,140,529,467,58,118,18,1,...,.385,.716,91,180,7,6,0,5,3,2016
5,LF,Brett Gardner*,32,148,634,547,80,143,22,6,...,.362,.713,92,198,6,8,4,5,0,2016
6,CF,Jacoby Ellsbury*,32,148,626,551,71,145,24,5,...,.374,.703,88,206,11,2,4,3,1,2016
7,RF,Aaron Hicks#,26,123,361,327,32,71,13,1,...,.336,.617,65,110,7,0,1,3,1,2016
8,DH,Alex Rodriguez,40,65,243,225,19,45,7,0,...,.351,.598,58,79,4,1,0,3,0,2016
10,RF,Carlos Beltran#,39,99,387,359,50,109,21,0,...,.546,.890,133,196,13,2,0,4,3,2016


In [155]:
# CLEAN 2017 same as 2016 dropped 'rk' and 9th row
df_2017.drop('Rk', axis = 1, inplace = True)
df_2017.drop([9], inplace = True)
df_2017['YEAR'] = ('2017')
clean_df_2017 = (df_2017[df_2017.index < 25])

clean_df_2017.head(10)

,Pos,Name,Age,G,PA,AB,R,H,2B,3B,...,SLG,OPS,OPS+,TB,GDP,HBP,SH,SF,IBB,YEAR
0,C,Gary Sanchez,24,122,525,471,79,131,20,0,...,.531,.876,126,250,9,10,0,4,1,2017
1,1B,Chris Carter,30,62,208,184,20,37,5,1,...,.370,.653,71,68,5,2,0,2,0,2017
2,2B,Starlin Castro,27,112,473,443,66,133,18,1,...,.454,.792,106,201,9,4,0,3,1,2017
3,SS,Didi Gregorius*,27,136,570,534,73,153,27,0,...,.478,.796,106,255,7,3,0,7,1,2017
4,3B,Chase Headley#,33,147,586,512,77,140,30,1,...,.406,.758,100,208,10,6,1,7,2,2017
5,LF,Brett Gardner*,33,151,682,594,96,157,26,4,...,.428,.778,104,254,4,8,5,3,2,2017
6,CF,Jacoby Ellsbury*,33,112,409,356,65,94,20,4,...,.402,.750,97,143,7,5,2,0,3,2017
7,RF,Aaron Judge,25,155,678,542,128,154,24,3,...,.627,1.049,171,340,15,5,0,4,11,2017
8,DH,Matt Holliday,37,105,427,373,50,86,18,0,...,.432,.748,95,161,14,3,0,5,0,2017
10,OF,Aaron Hicks#,27,88,361,301,54,80,18,0,...,.475,.847,122,143,8,3,1,5,0,2017


In [183]:
# In place in true on dropping row 9. dropped col 'Rk'
#df_2018.drop('Rk', axis = 1 , inplace = True)
#df_2018.drop([9], inplace = True)
df_2018['YEAR'] = ('2018')
clean_df_2018 = (df_2018[df_2018.index < 24])

clean_df_2018.tail()

,Pos,Name,Age,G,PA,AB,R,H,2B,3B,...,SLG,OPS,OPS+,TB,GDP,HBP,SH,SF,IBB,YEAR
19,OF,Shane Robinson,33,25,54,49,8,7,1,0,...,.224,.432,19,11,2,0,1,0,0,2018
20,OF,Clint Frazier,23,15,41,34,9,9,3,0,...,.353,.743,107,12,3,2,0,0,0,2018
21,IF,Adeiny Hechavarria,29,18,37,36,3,7,0,0,...,.361,.577,55,13,0,0,0,0,0,2018
22,OF,Jace Peterson*,28,3,11,10,0,3,0,0,...,.300,.664,86,3,0,0,0,0,0,2018
23,LF,Billy McKinney*,23,2,4,4,0,1,0,0,...,.250,.500,38,1,0,0,0,0,0,2018


In [158]:
#df_2019.drop('Rk',axis = 1, inplace = True)
#df_2019.drop([9],inplace = True)
#df_2019['YEAR'] = ('2019')
#clean_df_2019 = (df_2019[df_2019.index < 25])

clean_df_2019.head(10)

,Pos,Name,Age,G,PA,AB,R,H,2B,3B,...,SLG,OPS,OPS+,TB,GDP,HBP,SH,SF,IBB,YEAR
0,C,Gary Sanchez,26,106,446,396,62,92,12,1,...,.525,.841,119,208,3,9,0,1,3,2019
1,1B,Luke Voit,28,118,510,429,72,113,21,1,...,.464,.842,124,199,12,9,0,1,2,2019
2,2B,DJ LeMahieu,30,145,655,602,109,197,33,2,...,.518,.893,136,312,14,2,1,4,0,2019
3,SS,Didi Gregorius*,29,82,344,324,47,77,14,2,...,.441,.718,87,143,5,1,0,2,1,2019
4,3B,Gio Urshela,27,132,476,442,73,139,34,0,...,.534,.889,133,236,13,5,0,4,1,2019
5,LF,Mike Tauchman*,28,87,296,260,46,72,18,1,...,.504,.865,128,131,9,1,0,1,0,2019
6,CF,Brett Gardner*,35,141,550,491,86,123,26,7,...,.503,.829,117,247,6,4,0,3,0,2019
7,RF,Aaron Judge,27,102,447,378,75,103,18,1,...,.540,.921,143,204,11,3,0,1,4,2019
8,DH,Edwin Encarnacion,36,44,197,177,33,44,11,0,...,.531,.856,123,94,1,3,0,0,1,2019
10,MI,Gleyber Torres,22,144,604,546,96,152,26,0,...,.535,.871,128,292,10,3,1,6,3,2019


In [239]:
clean_seasons = [clean_df_2016,clean_df_2017,clean_df_2018,clean_df_2019]

seasons_combined = pd.concat(clean_seasons)



seasons_combined[['Age','G','PA','AB','R','H','2B','3B','HR','RBI','SB','CS','BB','SO','BA','OBP','SLG','OPS','OPS+',
                 'TB','GDP','HBP','SH','SF','IBB']] = seasons_combined[['Age','G','PA','AB','R','H','2B','3B','HR','RBI','SB','CS','BB','SO','BA','OBP','SLG','OPS','OPS+',
                 'TB','GDP','HBP','SH','SF','IBB']].apply(pd.to_numeric)
seasons_combined.drop('CS',axis = 1, inplace = True)
seasons_combined.describe()


,Age,G,PA,AB,R,H,2B,3B,HR,RBI,...,SLG,OPS,OPS+,TB,GDP,HBP,SH,SF,IBB,YEAR
count,95.000000,95.00000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,...,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000
mean,27.821053,68.60000,261.410526,232.294737,35.031579,60.042105,11.252632,0.873684,10.494737,33.578947,...,0.417063,0.728232,93.136842,104.526316,4.810526,2.284211,0.536842,2.073684,0.842105,2017.494737
std,4.068506,51.50282,222.000215,196.648237,32.580654,54.746595,10.428040,1.572653,11.100004,30.609267,...,0.171397,0.234605,59.572522,95.749212,4.520155,2.672409,1.165191,2.241970,1.552731,1.128675
min,21.000000,2.00000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,-100.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2016.000000
25%,25.000000,18.50000,55.500000,50.000000,6.000000,9.000000,2.000000,0.000000,1.000000,5.000000,...,0.348000,0.643500,71.500000,14.500000,1.000000,0.000000,0.000000,0.000000,0.000000,2016.500000
50%,27.000000,62.00000,208.000000,184.000000,25.000000,44.000000,9.000000,0.000000,8.000000,31.000000,...,0.425000,0.732000,93.000000,80.000000,4.000000,1.000000,0.000000,1.000000,0.000000,2017.000000
75%,30.000000,114.50000,460.000000,404.500000,60.000000,106.500000,18.000000,1.000000,17.000000,57.000000,...,0.493000,0.844500,123.500000,195.500000,8.000000,3.500000,1.000000,3.500000,1.000000,2018.500000
max,40.000000,158.00000,705.000000,617.000000,128.000000,197.000000,47.000000,7.000000,52.000000,114.000000,...,1.375000,1.875000,374.000000,340.000000,17.000000,10.000000,5.000000,10.000000,11.000000,2019.000000


In [58]:
driver = webdriver.Chrome(chromedriver)
driver.get()

In [17]:
driver.page_source[:1000]

'<html data-version="klecko-" data-root="/home/br/build" itemscope="" itemtype="https://schema.org/WebSite" lang="en" class=" js cookies cors history localstorage sessionstorage canvas csspositionsticky pointerevents unicode no-touchevents vibrate matchmedia flexwrap desktop is_live is_modern overthrow-enabled" style=""><head><script src="https://www.googletagservices.com/activeview/js/current/osd.js?cb=%2Fr20100101"></script><script src="https://rules.quantcount.com/rules-p-UeXruRVtZz7w6.js" async=""></script><script async="" type="text/javascript" src="https://sb.scorecardresearch.com/beacon.js"></script><script async="" type="text/javascript" src="https://hbx.media.net/bxl.js?cid=8CUFH1GPH&amp;dn=www.baseball-reference.com&amp;version=&amp;https=1"></script><script async="" type="text/javascript" src="https://secure.quantserve.com/quant.js"></script><script async="" type="text/javascript" src="//a.pub.network/core/pubfig.messaging.2.1.2.js"></script><script src="https://tagan.adligh

In [62]:
driver.quit()

In [59]:

caldo_feo = BeautifulSoup(driver.page_source, 'html.parser')


MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=62540): Max retries exceeded with url: /session/38e694420ee20f062f5167984b864f5a/source (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fdd16082e90>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [48]:
table = caldo_feo.body.find('table', id= 'team_batting').find('tbody')
table
rows = [row for row in table.find_all('tr')]
rows




#div_id_all_team_batting = body.find('div',id='all_team_batting')

<tr><th class="right" csk="1" data-stat="ranker" scope="row">1</th><td class="left" csk="1" data-stat="pos"><strong>C</strong></td><td class="left" csk="Sanchez,Gary" data-append-csv="sanchga02" data-stat="player"><a href="/players/s/sanchga02.shtml">Gary Sanchez</a></td><td class="right" data-stat="age">24</td><td class="right" data-stat="G">122</td><td class="right" data-stat="PA">525</td><td class="right" data-stat="AB">471</td><td class="right" data-stat="R">79</td><td class="right" data-stat="H">131</td><td class="right" data-stat="2B">20</td><td class="right iz" data-stat="3B">0</td><td class="right" data-stat="HR">33</td><td class="right" data-stat="RBI">90</td><td class="right" data-stat="SB">2</td><td class="right" data-stat="CS">1</td><td class="right" data-stat="BB">40</td><td class="right" data-stat="SO">120</td><td class="right" data-stat="batting_avg">.278</td><td class="right" data-stat="onbase_perc">.345</td><td class="right" data-stat="slugging_perc">.531</td><td class